<a href="https://colab.research.google.com/github/grantinator/ucla-research/blob/master/231N_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision


In [0]:
torch.eye(3) #identity tensor 3x3
torch.rand((4,3)) #4x3 random matrix/tensor

tensor([[0.0669, 0.1111, 0.9524],
        [0.5916, 0.3217, 0.7286],
        [0.2404, 0.0311, 0.7185],
        [0.4420, 0.9608, 0.2907]])

In [0]:
X = torch.rand((5,3))
#Get size/shape of X
X.shape

torch.Size([5, 3])

In [0]:
#Matrix Multiplication @ symbol
X.t() @ X

tensor([[1.4478, 1.4560, 1.5235],
        [1.4560, 1.9385, 2.0219],
        [1.5235, 2.0219, 2.1231]])

In [0]:
#Basic operations are defined as functions, for example add 1 to each element
#Add a dash after function to do operation in place as in A.add_(1)
A = torch.eye(3)
A.add_(1)

tensor([[2., 1., 1.],
        [1., 2., 1.],
        [1., 1., 2.]])

Tensor objects have a RequiresGrad member which is a boolean set to False by default, which is a switch determining whether or not you can calculate the gradient.
If this is selt to False and try to calculate gradient with .backward(), will get a runtime error.

In [0]:
A.RequiresGrad = True
A.backward

<bound method Tensor.backward of tensor([[2., 1., 1.],
        [1., 2., 1.],
        [1., 1., 2.]])>

**Cuda** 
Allows you to move a tensor object to the GPU to make calculations faster.
You can identify if you have a GPU with code below,

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device #If you have GPU output should be device(type='cpu')

#.cuda() allows you to put tensor in form that it will be computed in GPU
A.cuda()

device(type='cpu')

# Transforms
Common image transforms can be composed

In [0]:
#Random crop of image 
IMAGE_SIZE = 224

image.RandomCrop(IMAGE_SIZE)

#Sequential transforms
trans = transform.Compose([
  image.RandomCrop(IMAGE_SIZE)
  image.RandomHorizontalFlip()
  transform.ColorJitter(.3, .3, .3)
  transform.ToTensor() #Converts image to a tensor   
])



# Data Loader
Divide the dataset into small batches, and go through each batch and compute the gradient, then update weights.

epoch - one forward and one backward pass of **all** the traning examples

batch size = number of training examples in one forward/backward pass



Allows to get batches of data and process it at once and returns a patch. 
Each image transformation/processing is calculated in parallel.
Expects a tensor object, so as above should be converted to tensor.

Dataloader will provide you with iterable that allows you to create batches.

In [0]:
train_dl = DataLoader(train_ds, bath_size = 2, shuffl=True, num_workers = 4)
train_iter = iter(train_dl)
 
train_ds = SomeImages('file/path', transform = trans) #transforms defined above


class DiabetesDataset(Dataset):
  
  #Initialize data, download etc
  def __init__(self):
    #Download read data etc
    xy = np.loadtxt('csvfile')
    self.x_data = torch.from_numpy(xy[:,0:-1])
    self.y_data = torch.from_numpy(xy[:,[-1]])
  def __getitem__(self, index):
    #Return one item of the index
    return
  def __len__(self):
    #Return data length
    return
  
dataset = DiabetesDataset
train_loader = DataLoader(dataset = dataset,
                         batch_size = 32, 
                         shuffle = True,
                         num_workers = 2)


# Sampler
Defines how to sample from the dataset

1.   SequentialSampler
2.   RandomSamples
3.   SubsetSample



K Nearest Neighbor

1.   Training - essentially memorize dataset
2.   Testing.- just match most similar image


# Manhattan Distance L1
Measure of similarity Between Images


1.   Take one image and its pixels values (predicted) and subtract off elementwise each pixel (absolute value) and sum of each one.

L1 Depends on the coordinate system being used. 

*If your vector's elements have unique meanings probably use L1 Norm* 

K Nearest Neigbhor is basically just for a given image, compute manhattan distance for each image in testing set and find one with the smallest distance to be your prediction. Take this from K of the nearest neighbors, and take a majority vote from them to classify the point. 


# Euclidean Distance L2
Instead of sum of absolute value between pixels, could use the square root of the sum of squared differences between the pixels. 

L2 (And L1) distance doesn't account for qualititative differences between images. Such as an image vs itself shifted to the left by two pixels may have same L2 value as an image shifted to have blue hue vs original image.


# Hyperparameters
Examples are things like K, or distance formula. Things that aren't learned by the model but are a factor for the training.

# Curse Of Dimensionality
The training points much thoroughly cover the space. Since you are sort of dropping paint based on the color of the nearest pixels. In 1D, maybe you need 4 points to cover the space, in 2D you need 16 and in 3D you need 64. There is exponential growth in the required amount of data which is bad.

# Linear Classification

Neural Networks are sort of like lego blocks where you can stick together pieces. 

Linear classifiers are a common block that is used.

# Parametric Approach

Classifier now takes in an image, and a set of weights, which will spit out a number that corresponds to the probability of a guess. 

The paramaters, W, summarize the training data to be used at test time. 


$$f(x,w)$$ Goal is to find/construct the function f()


---

Image 32x32x3 numbers (3072 total) --> $$\underset{10 \times 1} {f(x,W)} = \underset{10 \times 3072} {W} \ \underset{3072 \times 1} {x} --> \underset{10 \times 1} {y}$$ is a vector containing rankings for each of the ten labels. 

May add a bias term to the function, for example if data was imbalanced (had many more cats than dogs) you could give preference to the dog weights.

If you were to look at weights for each category and reverse them to get rought idea/image of the pattern being recognized they would not look like any one image. Sort of an average of many. This is because it can only learn on one template.


**Properties of W**

1.   Each row of W is a weight relating to the value of how much each pixel contributes to the probabilitity of a class
2.   Once a loss function is defined, perform gradient descent to find the W that minimizes the loss function. This is how W is found.
3.   A scaled factor of W will not change the loss. I.e., if W has loss 0, then 2W also has loss 0. 

**Each row of W** can be thought of as a template for one of the classes. Since each row of W is like a classifier for one of the classes (since if you take in a 3072 dimensional vector representing an image and want to output one of 10 classes, W will be 10 x 3072). So the score for each class of an image is found by comparing the input X to each template. 






# Support Vector Machine Loss

Lec 3 17:31

Sum over the scores for the incorrect categories (Sj), as in if it is a picture of a cat and your score for dog and bird are 5 and 3 respectively, this sum will be 8. If the score of your correct category Syi is greater than the score of your incorrect category plus some safety margin, say 1, then the contribution for this point to the loss will be 0. 

$$ S_{yi} \geq S_j + 1\ then\ add \ zero\ to\ loss$$

If the sum of scores for incorrect categories + 1 is greater than the score for correct category, then your penalty will be the sum of scores for the incorrect categories minus the score for correct category plus one. 

Formalized as a sum over all images in one category,

$$\sum\limits_{j \neq y_i} \begin{cases} 
                                                 0 & \text{if $S_{yi} \geq S_j + 1$}\\
                                                 S_j - S_{yi} + 1 & \text{otherwise} 
                                                 \end{cases}$$
                                                 
Idea behind this loss function is that you are "happy" if the score for the true category is much higher than the score for any other category. 

Basically if another incorrect label is scored higher than the correct label, it will contribute to the loss of amount score incorrect + score correct + 1. Perform this for each incorrect label in the category. Then perform this on each test image in the cateogory. Then perform on each image category in the test dataset. The loss will be the average of the sum of every score.

If every score is around 0, then the loss should be approximately # classes - 1. This should be the value of loss for initial evaluation of loss - useful for debugging. 

Note you could also square the value of the loss, in which case you are suggesting that very bad errors are "squared bad" in penalty, whereas in the regular loss a loss at any magnitude + a is always loss + a. 

# Softmax Loss

The scores are used to compute probability distributions over each class. The idea is to compare the computed probability distribution to the real probabilitity distribution. The real probability distribution will have all the mass on the true class and zero elsewhere, so we encourage 

Once they are probabilities (all of which sum to one), then the loss will be the **negative log of the true class's probability score**. If your classification is totally correct (i.e. score for right class is positive infinity and score for wrong classes are negative infinity) then your loss will be zero. The maximum loss is unbounded. 

Note that the softmax function itself is a function that takes a vector of K entries and normalizes it into a probability distribution consisting of K probabilities. In the original vector some values may be negative or greater than 1, and dont necessarily sum to 1. After applying softmax, each value will be in the range (0, 1) and will sum to 1. 

Differences in probability from soft max vs true label (1 - hot encoded) is computed by - true label * log(predicted probability). For one hot encoded, only one true Y will be 1, rest will be 0. Therefore, you will only get maybe one computation in this for the value where true y = 1. This is onyl true for one hot encoding.

$$
  y_k = \frac{\exp(\phi_k)}{\sum^{c}_j \exp(\phi_j)},
$$

While SVM loss will aim to get the data point over the correct bar (where score of wrong is less than score of right) and then give up, because regardless of scores after this point the contribution will be zero. 
Softmax will continue to push the score for the correct label to infinity, and the score for the wrong labels to negative infinity. 

# Batch Gradient Descent (Stochastic Gradient Descent)

Evaluating the loss for gradient descent could be very slow if is N is say 1 million. Instead, at each iteration of gradient descent, sample a batch of data points to evaluate the loss function. Then update parameters based on gradient descent for this. 

# Feature Vectors

Often times it is worthwhile, in case of things like multimodality, to create "feature vectors," which describe/summarize a feature in order to make the image linearly classifiable. This is as opposed to just giving the classifier raw pixel data. 

For example, maybe come up with a color histogram by dividing a hue slider into buckets, and then creating a histogram of the number of pixels that fall into each bucket. Similarly, divide an image into its edges and create a histogram of the edges. Create feature vectors that represent that data.

In a convolutional neural network, you are training the feature recognition and the classifier that uses the feature vectors.

# Convolutional Neural Networks
Lec 5

**Fully Connected Layer** 
If you have a 32x32x3 image, you stretch it out to 3072 x 1 vector and multiply it by a weight matrix that is 10x3072 to get scores for one of 10 labels, and apply the an activation to each neuron in the 10x1 output vector. 

**Convolutional Layer**

Goal is to preserve spacial structure, so cannot stretch it out into one long vector. 

Now, the weights are small filters, say 5x5x3, and you slide this filter over the image and compute dot products over every location in the image. 

**Filters**

The filters always extend the full depth of the input, ie always depth 3. 

You apply the filter to some chunk/small 5x5x3 area of the image andd perform a dot prdouct. So now you turn the 5x5x3 into one vector as before, and computer the dot product + bias. 

$$\underset{1\times75}W^T\underset{75\times1}X + \underset{1 \times 1}b = \underset{1 \times 1} {scalar}$$

Common filter sizes are 5x5 or 7x7 

Common number of filters are powers of 2, e.g. 32, 64, 128.


**Moving the Convolution Filter**

Start at the upper left corner, and for each picture in the image, slide the filter so it is centered around every pixel.

Doesn't necessarily have to be at every pixel, could slide by two input pixels at a time for example. 

You may have multiple filters (little convolution boxes) where each filter looks for a specific type of feature. This will result in multiple activation maps, one for each filter.

**Activation Map**

This is the value of the convolution filter at every input location. 

You could have multiple activation filters, so you convolve and put 6 convolution filters onto activation map to get 6 activation maps. 

Say activation map is 28x28x1, if you have 6 if this your activation map will be 28x28x6. 

The height and width of an activation map are the number of "slides" that fit over the image. As in, if your convolution filter can slide five times over the image horizontally and five times vertically, your activation layer will be 5x5x1. Technically this is a function of the **stride**, but the dimension of the output is how many convolutions you fit on the original image. 

The stride must fit the image size though.

**Output size = ((N - F) / stride) + 1, N = dimension of 1 side of image, F = dimension of one side of convolution filter**

In practice, it is common to pad the border of the image with 0's which allows the stride size stuff to work out so the output size matches the input size. If you dont zero pad, the size of your activation maps will shrink fairly fast with each successive convolution. 


The depth of an activation layer tells you how many filters were used to obtain it. 

***A ConvNet is a sequence of the convolutional layers interspersed with activation functions in between***

This results in a hierarchy of layer/activation maps. Since each layer has multiple filters, and each layer has its own activation map. So you have a heirarchy where the low level/start features find basic features, and higher level layers will be finding more complex features like blobs or maybe eyes/mouths. 


Note on brain/neuron view: each neuron is now looking at a local space of the input image (as opposed to all the data) so now you are seeing how much each neuron is triggered at each location in the image. 





# Non Convolutional Layers

## Pooling Layers
Pooling layers make the representations smaller and more manegable. 

These spacially (i.e. not depth) downsamples a given activation layer. 

A common way to do this is max pooling. So you have activation layer of 4x4 and take a 2x2 pooling filter. This divides the original activation layer into 4 2x2 squares, **max pooling** will take the max value from each of these squares and put it into a smaller 2x2 square. Unlike the convolution layer, here the filters have no overlap. 

Max pooling is commonly used because each score in activation layer is an activation of the neuron, so max pooling is in some sense representing how much did this neuron fire at any location in the image. I.e. if it is a filter for eyes, the max value might tell you if it happened anywhere in the image. 

**Pooling Layers**


---

accepts a volume of size $$W_1\times H_1 \times D_1
$$

Requires three hyperparemeters:
$$\text{Spacial extent}\ F,
\text{the stride}\ S $$

Produces a volume of size

$$W_2 \times H_2 \times D_2 \text{Where}\\
W_2 = \frac{(W_1 - F)}{S} + 1\\
H_2 = \frac{(H_1 - F)}{S} + 1\\
D_2 = D_1$$

Common settings are 

F = 2, S = 2

F =3, S = 2

---

## Fully Connected Layer

Final layer, which takes the output of the convolutional network output which will have some volume width x height x depth and stretch it out into a N x 1 vector. So now you are getting rid of the spacial structure. Now you have 1d input and apply a fully connected layer (connected to each entry in the vector) and output some sort of scores. So now you are combing all the data to come up with outputs. 



# Training Neural Networks

## Activation Functions
At any given layer, you multiply by the weights of the convolution filter and then pass the output through a non-linearity. These non-linearities are activation functions.

**Types**


*   Sigmoid
    - Squashes numbers to range [0,1]
    - has nice intepretation as a 'firing rate' of a neuron
    - **problems** saturated neurons can 'kill off' a neuron. This is because at very positive/negative values of the sigmoid function, the gradient/slope is near zero. So any value passed down in the gradient will be multiplied by 0 and thus kill the gradient.
    - **problem** outputs are not zero centered. If the input to a neuron is always positive. Essentially, if x is always positive, the gradient will be the sign of the upstream gradient coming down as df/dx will be all positive. 
      - this restricts the potential directions for the gradient to go. You may have to go in a zig zag/inefficient path to minimum. 
    - this is why you want zero mean data. 
*  tanh
    - squashes to range [-1, 1], so it is now zero centered.
    - still kills the gradient when you are in the 0 range (x is large in magnitude)
* ReLU f(x) = max(0,x)
    - if x is less than 0, passes 0, if x is positive it will pass through x
    - Does not saturate in positive regeion, but will kill the gradient in half of the domain
    - Computationally efficient
    - More biologically plausible than sigmoid
    - **Problem** not centered at zero
    - **Problem** in theory you could initialize the weights that result in the ReLU always being 0 and so you will never learn because it always kills the gradient
        - even with good initialization, you could have some percentage of the training data that results in the ReLU killing the gradient
* Leaky ReLU f(x) = max(0.01x, x)
    - Solves a lot of the problems with normal ReLU as it will not saturate at any area.
* Paramatric Rectifier ReLU (PReLU) f(x) = max(ax, x)
    - same as leaky ReLU
* Exponential Linear Units
$$f(x) = \begin{cases} 
                                                 x & \text{if $x > 0$}\\
                                               \alpha(exp(x) - 1)& \text{if $x \leq 0$} 
                                                 \end{cases}$$
                                                 
    - All benefits of ReLU
    - Closer to zero mean outputs
    - negative saturation domain compare to Leaky ReLU which adds robustness to noise
    - **problem** computationally expensive computes exp()

**Main Result** is that there are a lot of modifactions on ReLU which can be experimented with. 

* Maxout Neuron
    - takes the max of ReLU and Leaky ReLU
    - generalizes ReLU and Leaky ReLU
    - Linear Regime. Does not saturate. Does not die. 
    - **problem** doubling the niumber of parameters per neuron because each neuron now has w_1 and w_2
    
$$max(w_1^Tx + b_1, w_2^Tx + b_2)$$


**Main Result** Use **ReLU** or some variant for experimentation. Can try tanh but dont expect much. Dont use sigmoid. 


## Data Preprocessing

Generally want to zero center the data for the purpose of non-zero centered activation functions. Also want to normalize the features so all data is in the same range, i.e. so units of variables dont matter. This isn't really an issue for images, though. 

**TLDR** For images
* subtract mean image (mean image = [32, 32, 3] array)
    - Do so at training and test time with the one mean image
* subract per channel mean (mean along each channel = 3 numbers)

## Initializing Weights

* For small networks can sample from normal distribution with small standard deviation. 
    - does not work for deeper neural networks
    - Eventually the activations will all be 0, because you end up multiplying activations (centered at 0) by small initialized weights so the distribution becomes centered around 0 with 0 std dev
    - Will result in the gradient update values being very small so W is essentially never updated
* cannot used std. dev 1 as the likely the activation function will be saturated because the resulting dot products will be large. 

**Xavier Initilization**
Initialize W to be a sample from standard gaussian and then scale by the number of inputs you have. This will result in the variance of the input being the same as the variance of the output. 

If you have a small number of inputs, you will have larger weights, and if you have small number of inputs you need larger weights to have larger variance at output. Vice Versa for large number of inputs.

Does assume that you are in the active region of the activation function. So when you are using ReLU, because ReLU is setting approx half the units to zero at each time, you are in essence halving the variance. This will result in output variance being too small so breaks the Xavier method. You can account for this by, when dividing each weight by the input size you divide that by 2. 

e.g. W = norm.sample() / sqrt(num_inputs/2)

## Batch Normalization

Goal is to have unit gaussian activations, so you just force them to be. 

 This is a differentiable function that you can perform backprop on.

$$ \hat{x}^{(k)} = \frac{x^{(k)} - E[x^{(k)}]}{\sqrt{var[x^{(k)}]}} $$

If your input data has N training examples in each batch, and each batch has dimension D. Compute the mean and variance for each dimension D, i.e. each feature element. 

This is typically done after fully connected or convolutional layers and before nonlinearity. On convolutional layers, you want to normalize jointly accross all of the spacial locations in the activation map. So on convolutional layers will have one mean and one variance per activation map, and will do this across every example in the batch. 

You then perform some affine transformations witht the following learned parameters gamma and beta. This allows you to be able to recover the original values if you want to as the network could learn gamma to be the mean and beta to be the variance. Purpose of this is that, although in general its a good idea to force the data to be a unit gaussian, it may not always be so you allow for gamma and beta to exist to alter it slightly. For example in tanh a unit gaussian may not be ideal for the values less than 0.

Batch normalization is applied to individual neurons which are the inputs to each layer.

$$y^{(k)} = \gamma^{(k)}\hat{x}^{(k)} + \beta^{(k)}\\
\text{Note the network can learn} \\
\gamma^{(k)} = \sqrt{var[x^{(k)}]}\\
\beta^{(k)} = E[x^{(k)}]$$

**Benefits**
    - improves gradient flow through the nextwork
    - allows for higher learning rates
    - reduces the dependence on initialization
    - acts as a form of regularization, because it ties the inputs of a bath together
    
**At test time** the mean and variance are not recomputed.

## Finding HyperParameters

**First Stage**: only a few epochs to get rough idea of what params work
**Second Stage**: longer running time, finer search

**Tip** if cost ever > 3 x original cost break out of loop and figure out whats wrong (probably learning rate).

Explore learning rates as 10^range of values. Shouldd sample randomly rather than in some organized gridlike fashion between two hyperparameter options.


## Fancier Optimization Methods
### Problems with Stochastic Gradient Descent

**Note** stochastic gradient descent is normal gradient escent performed on minibatches. 

Potentially the loss could look like a 'taco shell', where it is very flat in one direction but very sloped in another direction. You will likely zigzag towards the minimum with normal stochastic gradient ddescent which is not ideal.

**Saddle points \ local minimum** the gradient can easily get stuck. In higher dimensions saddle points become a much larger problem.

### SGD + Momentum

Gradient builds up velocity as a running mean of gradients.

Rho gives 'friction' typically rho = 0.9 or 0.99

$$v_{t+1} = \rho v_t + \nabla f(x_t)\\
x_{t+1} = x_t - \alpha v_{t+1}$$

For saddle points, even though the gradient at this point might be close to zero, the built up velocity from rolling down hill will hopefully get you through the saddle. 

**Nesterov Velocity** Take a step in direction of the velocity and then evaluate gradient at the point after velocity step. Then go back to start point and go in direction of the mix of the two. 

$$v_{t+1} = \rho v_t + \nabla f(x_t + \rho v_t)\\
x_{t+1} = x_t - \alpha v_{t+1}$$

### AdaGrad

Over course of optimization, keep a running estimate of sum of squared gradients. When you update parameter vector, divide learning rate by the square root of the squared gradient. Since the squared gradients are monotonically increasing, your step size will decrease no matter what. **Better to use RMSProp**

## Adam

Combine the momentum and RMSProp ideas. Maintain an estimate of the first moment (momentum) and second moment (squared gradients). When you make a step, you multiply buy momentum and divide by squared gradients.

However, in above method at the first time step, you will be dividing by a very small number and therefore taking an improper large step for your first step in gradient descent. 

Two bias correction term are created to correct for this.

**Adam is generally a good default approach for optimization** To set beta1 =0.9, beta2 = 0.999. 

### Learning Rate

To pick learning rate you can decay the learning rate over training with

$$\alpha = \frac{\alpha_0}{(1 + kt)}$$

When the loss starts to plateau you could drop the learning rate even small. This is like once you get into a good local minimum you take smaller steps to get a more refined local minimum.

Generally experiment with learning rate decay later, get everything else sorted first.


# Improving Performance on Unseen Data

### Model Ensembles
1. Train multiple independent models.
2. At test time, average their results.

Fairly common to get maximal performance. 

Instead of using multiple models, you could keep multiple snapshots of a single model during training and then average the results of these snapshots.

Unfortunately, this still requires running multiple at test time. Ideally, you would like to have one very good model.

### Regularization

* Dropout
   - everytime you do a forward pass through the network, you randomly set some neurons to zero. In some sense you are 'sampling' a neural network and only updating the weights of the sampled neural network
   - Very useful and simple to apply
   - Sort of like doing model ensambling within a single model
   - **when prediction you should scale your weight matrix by the dropout probability**
* Data Agumentation
   - Randomly alter the input data(images)
   - Common are translations, random crops, color jittering (change contrast or brightness), 
   - At testing you average out with a fixed set of crops, i.e. test on the 5 same crops of each image.
* Drop Connect
   - Randomly zero out some weights 
* Fractional Max Pooling
  - Normally when you do pooling, you pool over some, say 2x2 window of data, you make this pooling window variable
* Stochastic Depth
  - Randomly drop layers in training.
    
Dropout is one instance of regularization where at training you introdduce some randomization and testing you average out the randomness. 

In many cases batch normalization should be used and is enough, but the methods mentioned above can be used if your model is still overfitting. 

### Transfer Learning

In theory you need a large amount of data to train a CNN. If you only have a small amount you will likely overtrain. Transfer learning is a solution to this where you train the model on a huge dataset, and then on the last fully connected layer that does the classification, you reinitialize the matrix and train only that and freeze the weights of the previous layers. This works pretty well (what did in flower recognition). Note that this could be extended to update the layers before the final fully connected layer as well. 


# Recurrent Neural Network

With characters, the idea is that the network is just trying to predict the next character in a sequence. 

## Many to One

There is an RNN cell which has some hidden internal state which is updated every time data is fitted. The updates of the hidden state are a function of the data and previous hidden state. Think about it as previous hidden state and current data x go into function f() to compute the next hidden state. Then this next hidden state and new data x go into f() to compute a third hidden state. Common among all of these is a W weight matrix involved in f(). Note that each hidden state you could use to predict a yi for a one to many output. 


## Many to Many
You want to convert a sentence  in english to in french
You have an encoder which is like a Many to One that takes the English sentence. At the end of this you have a One to Many that takes the output of the encoder and gives the sentence in french. 

**Backpropagation through time** Need to see the way the net work trains and tests. Based on this, if you were to backpropogate normally, you would have to do a forward pass, then backprop one layer, then do another forward pass since the input to the next layer depend on the output of the previous layer. Instead, you break the backprop into chunks, so you do a forward pass through some N layers, then back prop, then move to another chunk.

So far all of the notes have been on classifying an image to one of some number of given classes. However, there are other tasks that can be performed.

# Segmentation

For every pixel in an image, output whethere the pixel is grass, cat, tree, or sky. Note that semantic segmentation does not differentiate instances. So if you have two cows next to eachother (such that their pixels connect) you will just get one big blob of cow pixels. 

**Approach** Could apply. a fully convolutional network, which is just a stack of convolutional layers with no fully connected layer. Here, each convolutional layer preserves the size of the input. It then outputs a tensor of scores the same size as the input, where each entry in the output is a classification of that pixel. 

One issue above is that, since each convolution preserves the size of the image, it is very expensive to run many convolutions.

**Better approach** You do some convolution, and then down sample the feature maps through max pooling. Then, for the second half of the network you are trying to upsample, or increase the size of your activation maps. 

**Upsampling methods** 
* Unpooling 
   - Say you have a 2x2 grid that is 1 2 3 4 and want 4x4 grid. Then upper 4 points are just all ones, upper right 4 points are all 2's etc.
   - Bed of nails is where you just take the value in the 4x4 grid and create a 4x4 grid where, say 1, is surrounded by 3 zeros. Do this for every value to get a 4x4 grid.
* Max Unpooling
  - Remember which element was max, and note that these maxes will have changed later in the network
    - say a 5 was your max for the upper left square of a 4x4 grid, and it came from the bottom right corner.
    - Now your max unpooling values are 1 (where the 5 was put in the max pooling), so you put a 1 where the 5 came from.
* Transpose Convolution
  - Unlike the above methods, which are just fixed functions, Transpose Convolution is a learnable layer that lets you do learnable upsampling.
  - This is sort of like a convolutional layer but it does upsampling and learns the weights to get the desire results from upsampling. 
  - Now you are going from a 2x2 region and you want a 4x4 region.
    - To do this, instead of taking an inner product like normal convolution, you take the value from the 2x2 feature map and multiply the filter by the scalar value, and then copy thos features to the output.
      - So the output is weighted copies of the transpose convolution where the weights come from the activation layer.
      - Where the fields overlap in the output, you just sum the outputs
  
The purpose of this is, when you are doing semantic segmentation you want the classification to be pixel perfect, as in the pixel boundaries to be very detailed. When you are doing max pooling you lose some of that spacial information in order to lower the dimension of the information, so you are sort of trying to recover the lost information. 


# Classification + Localization

Now you want to assign a category label to the image, **and** you want to draw a bounding box around the object in the image. 

Here you assume there is one instance of the object in the image. 

Now you use CNN machinery like before, but you now output the probability of each class **AND** a set of bounding box coordinates. 

Now you have two loss functions, your normal softmax loss and a measure of dissimilarity between the true box and the predicted box. Typically this is an L2 loss. Now you have a scalar to weight each loss and then sum them based on this weighting. This weighting hyperparameter can be difficult to set because you cant use loss as reference to check the quality of the hyperparemeter.

# Object Detection

Everytime a category/class appears in an image, you want to draw a box around it and label it. The number of classes that appear in an image could be variable which makes it difficult.

**Sliding Window Approach** Take different crops of the input image and feed each through the CNN which will make a classification on the crop. The network now has another label called background, meaning there is no object there. 

Unfortunately, to really sample an image you would have to classify thousands of crops which is very expensive. Therefore this method is rarely used.

**Region Proposal Approach**  The region proposal is fairly fast method (not a neural network method) and return proposal regions where an object might be found. This is fairly fast, but many of the proposed regions are not actual objects. However, if an object exists it will probably be covered.

Now you can pass the proposed regions to a CNN, which will be much faster than testing all possible crops. Note you need to change the different sized regions to the same size for the CNN. 

Your CNN will now predict whether or not there is a label, and an updated 'better' box to draw around the object. This is called an offset. Note that the offset could be outside the region of interest. Say a region of interest cuts off someone's head, but your CNN knows that people usually have heads so it proposes a box that is slightly higher. 

**Problems** Still fairly computationally expensive, as you still may have ~ 2000 region proposals. Overall fairly slow. 

**Fast R-CNN Modification** Instead of running each region proposal seperately, you run the entire image. Then you project the region proposals onto the convolutional feature map, and crop from the convolutional feature map. So for the first layers of the network you are performing on the whole image. 

Once you crop from the activation layer, you still need to transform the images to same square size through a ROI Pooling method. **R-CNN is much faster (10x faster to train, test time is very very fast)**

**Faster R-CNN** In Fast R-CNN the bottleneck on speed was computing the region proposals (which is very fast but still a bottleneck). Now you insert a region proposal network (RPN). So you have 4 losses now: 1. RPN classify object/not object, 2. RPN Regress the bounding box, 3. Final classification, 4. final box coordinates. 







# Whats Going on in a CNN

## First Layer
Since the first layer is a dot product between the convolution filter and the actual image, these can be somewhat directly intepreted. Generally these might be detecting oriented edges (bars of light and dark at various angles) or opposing colors.  These are showing which areas in the input are 'exciting' the convolutional dot product the most.

## Later/Middle Layers

These layers/activation maps are much less intepretable. This is because you are seeing which inputs 'excite' the dot product the most, however the input is a previous layers output. 

The activation maps are somewhat interpretable. This will give you a sense of what types of features that convolutional layer is looking for. 

**General note**: an input has multiple convolutional filters run over it. This outputs a convolutional volume of depth equal to the number of filters. A slice of this volume is an activation map. Each nueron corresponds to a small subset/patch of an image under a certain activation map. A neuron is one scalar value in an activation map. 

## Last Layer

Could run a bunch of images through a CNN and record the 4096 dim vector output and try to intepret those. The nearest neighbors to these 4096 vectors are typically similar to the test image. This is nearest neighbors in feature space. Unlike nearest neighbors in pixel space, where the images are maxed pixel wise, so a white dog may be matched to a white wolf. In feature space matching, the content of the images are matched but not necessarily exactly pixel wise. 

t-SNE is a method to give some 'location' to the 4096-d vector. One visualization method is to keep your original image, compute the 4096-d vector to get a t-SNE location, and then map the original image to the t-SNE location. What you will see is that similar semantic images will be grouped.

## Saliency Maps

Tell you, for each pixel if you were to wiggle that pixel a small amount, how much the prediction would change. This will show you the most important pixels for the prediction. 

## Guided Backprop

Compute the gradient of a neuron with respect to a pixel. Will tell you what type of input will cause the neuron to fire. 

## Gradient Ascent

Change the pixels of a randomly initialized input image to maximize a given neuron. Then use the gradient of the neuron w.r.t. the pixels to update the pixel values of your image. 

$$I = argmax f(I) + R(I)\ \text{ where}
f(I) \text{ is the nuron value}\ R(I) \text{ is a regularization to make the image look like an image}$$

Often times the regularization is simple L2 norm. If you dont add the regularization, the image will still maximize that neurons value but it wont look like much of anything.

**Improving Regularization to Improve the Generated Images** 
1. Add gaussian blur to image
2. Clip pixels with small value to 0
3. Clip pixels with small gradients to 0

## Fooling Images/ Adversarial Examples
1. Start from an arbitrary image
2. Pick arbitrary class
3. Modify the image to maximize the class
4. Repeat until the network is fooled

So you take an image of an apple as an elephant and tell network to classify it as a koala. Then keep updating the image until the network is convinced its a koala.

## DeepDream 

Another use of gradient ascent to change images.

Take some image and a layer in a CNN
1. Forward: compute activations at chosen layer
2. Set gradient of chosen layer equal to its activation
3. Backward: compute gradient on image
4. Update image






 



# Supervised vs Unsupervised Learning

## Unsupervised learning
Have unlabeled training data and you want to learn the structure of the data. For example, estimating the underlying distribution of the data. 

In supervised learning you try to learn a function to map x -> y.

In unsupervised learning you try to estimate the underlying structure of the data. 

## Generative Models
Given training data, generate new samples from the same distribution. 

## Generative Model 1 

### Explicit Density Moel

p(x) = product p(x_1 given x1 thru x_i-1) so likelihood of image is the product of the probability of each pixel.
$$p(x) = \prod {p(x_i|x_1, x_2...x_{i-1})}$$

Need to define what 'previous pixels' means - this depends on the approach.

#### PixelRNN

Start from the pixel in the upper left hand corner, and move one out (one down one right). Dependency on previous pixel is modeled using an LSTM. This is sequential generation, however, so it is slow. 

### Pixel CNN

Still generate pixels starting from the corner, however the dependency on previous pixels is now modeled using a CNN. You apply a CNN over a context region (like a frame) that is centered around the pixel of focus. At each location you are output a distribution of pixel values. Want to maximize the likelihood of the training data being produced.

This method of maximizing the likelihood is faster because you have the true value of the pixel in the training data. However, generation time is still slow.

## Generative Model 2

### Variational Auto Encoder
Define intractable density function with latent z, meaning you cannot optimize directly so you have to compute a lower bound. 

**Autoencoders** Have some input data and you want to learn some featuers z. Have a function f that maps the input data to a set of features. 

Typically, z reduces the dimension (it is smaller than x). This because z should capture only the most important parts of x. 

To train this, you also create an encoder that reconstructs the original data. This can be compare to the input as a measure of loss. 

An L2 loss function is used so you just compare the original and reconstructed data pixelwise. Once the model is trained, you get rid of the decoder. Then you use the encoder and attach a classifier network on top to output a class label. Even though the training data was unlabeled, you were able to train the first part of the network on untrained data. Now you could apply some labeled data for the final part of training.

## Variational Autoencoders

Have an encoder network that outputs a mean and covariance (z) matrix given data (x). Then add a decoder network that outputs the man and covariance of x given z.

Based on the idea of autoencoders but allows you to generate new images. 

Assume the training data is generated form underlying unobserved (latent) represnation of z. For z, sample from a prior based on what you think the distribution for a feature is (maybe a normal distribution) then you sample from a likelihood p(x | z). p(x | z) is a much more complex function - this part will be represented with a neural network. 

$$p_{\theta}(x) = \int p_{\theta}(z)p_{\theta}(x|z)dz \\ p_{\theta}(z) \text{ Guassian prior} \\ p_{\theta}(x|z) \text{ output of network}$$

Essentially, the p(z|x) is undeterminable, but after some math yo can determine a lower bound. To train a variational autoencoder, you otpimize the lower bound. 

**Forward Pass** Take input data -> pass through encoder to get Mean of z given x and variance matrix of z given x. -> sample from the distribution of z given x and put this sample through a decoder network, to get mean of x given z and variance matrix of x given z. -> sampe of x given z to get x hat. 

**Generate Data** Sample from the true generative process to compute distribution of x given z and produce xhat. Note that the variation of z's elements (z1, z2...) are real factors like z1 = amount of smile and z2 = head pose. 

## Generative Model 3 
### GANs
Dont work with any explicit density function. Take a game theory approach to learn to generate from training distribution through a 2 player game. 

The training distribution is very complex high dimensional and there is no direct way to do this. Sample from a simple distribution and learn a transformation to the training distribution. 

**Structure** Input a vector of random noise, pass through a generator network and the output will be a sample from the training distribution. 

**Generator Network** will try to fool the network by generating images.

**Discriminator Network** Try to distinguish between real and fake images. This will be trained on images from the generator network and real training network. 

Idea is that if your discriminator network is good at identifying fake images and your generator network is good at fooling the discriminator, you are generating good samples. 

**Training** Alternate Between:
1. Gradient **ascent** on discriminator
2. Gradient **ascent** on generator being wrong as loss function
   - Optimizing on the generator does not work that well in practice, though
   - The slope of the loss for this function is steeper when the generator is doing well, however, when the generator is bad the slope will be flatter
   - As a result use gradient ascent 

You sample a minibatch of noise samples and a minibatch of real images. You pass noise samples through generator and pass both through generator. Note: these images are 'labeled' in that you attach a fake or real label. 

Then update the discriminator using gradient ascent.

Then sample a minibatch of noise samples and update the generator using gradient ascent. 

Once you train the generator and discriminator network, can just use the generator network to get new images. 

# Reinforcement Learning

Agent and environemnt where environment gives the agent a state and the agent takes an action and receives some reward from the environment. The Agent will learn to take actions to maximize its reward.

## Markov Decision Process

Formalization of this RL problem

**Markov Property**: current state completely characterises the state of the world.
Which is defined by

$$S \text{: set of all possible states}\\
A \text{: set of possible actions}\\
R \text{: distribution of reward given (state, action) pair}\\
P \text{: transition probability i.e. distribution over next state given (state, action) pair}\\
\gamma \text{: discount factor}$$

At t=0, environment will sample initial state, then for t=0 until done:
  - agent selections action at
  - environment samples a reward given action
  - environment samples next state
  - agent receives reward rt and next state st+1
  
Want to find optimal decision making policy for agent. However, you need to handle randomness like the initial state and the next state. So you maximize the expected sum of the rewards. 

**Value Function** How good is a sate? A value function is the expected cumulative reward from following the policy.

**Q-value Function** How good is a state action pair? The q-value function at state s and action a is expected cumulative reward from taking action a in state s then following the reward policy.

**Bellman Equation** given any state action pair the value of this pair is the reward you will get from this pair plus the value of whatever state you end up in. Since you know you have the optimal policy, you know you will get the maximal reward based on your action.

It is computationally expensive to evaluate Q*(s,a) at every time so you use a neural network to estimate the Q-value * function (Q-value under optimal policy).

Loss is intended to find optimal policy. The network that you are training takes input of the current state and will predict the Q-value function. 

The network will have some convolutional layers and then a fully connected layer to get Q-values. 
If you learn from consecutive states, these training samples will be correlated which slows down and adds bias into training. **Experience replay** solves this by continually updating replay memory table of transitions as experience episodes are played. In a game, these are random samples from the game states. 

Start by intializing replay memor and weights. Then initialize state using the starting game screen pixels. For each time step, with small probability select a random action and explore or otherwise take greeddy action. Execture action and observe reward and store the state and reward and image in replay. 

**Problem** q-value can be very complicated and you are trying to learn the value of every (state,action) pair. 

Goal is to find optimal policy to get the highest expected reward. This can be done via gradient ascent on the policy parameters.

#### Reinforce algorithm
Sample some rewards for a policy and find the expected reward for this policy over the sampled states and actions. Taking gradient of this expected reward is impossible so instead you take expectation of gradient after some math. If your reward for a trajectory (seq. of actions) push up the probabilities for all the actions you took. If the reward is low, push down the probability of all the actions you took.

Above means that, even though not every action that resulted in high reward is good, in expectation, this will average out. However, the variance of this assignment of this estimator is high. 

Introduce a baseline factor that is expected reward you would have gotten (moving average of all past rewards). This can be a scaling factor for reward got - expected reward * direction to push probabilities. 

# Coding

In [0]:
import numpy as np
from torch.autograd import Variable
## Linear Model
w = 1.0 #Random weight guess

# Forward pass of model
def forward(x):
  return x * w
#Function to calculate loss
def loss(x, y):
  y_pred = forward(x)
  return (y_pred - y)**2

#Simple implementation of gradient descent
x_data = [1, 2, 3]
y_data = [2, 4, 6]

def gradient(x,y):
  return 2* x *(x* w - y)

lr = 0.01

w = Variable(torch.Tensor([1]), requires_grad = TRUE) #Initialize to random value


for epoch in range(10):
  for x_val, y_val in zip(x_data, y_data):
    l = loss(x_val, y_val) #Compute loss
    l.backward() #backprop
    grad = gradient(x_val, y_val)
    w.data = w - lr * w.grad.data #Gradient from PyTorch Variable
    w.grad.data_zero() #Zero out gradient after descent
    
  print("progress:", epoch, "w=", w, "loss=", loss)
  
#Or using Pytorch more
x_data = Varaible(torch.Tensor([[1.0], [2.0], [3.0]])) #3x1 data
y_data = Varaible(torch.Tensor([[2.0], [4.0], [6.0]]))

class Moddel(torch.nn.Module):
  def __init__(self):
    #In constructor instantiate two nn.Linear module
    super(Model, self).__init__()
    slef.linear = torch.nn.Linear(1,1) #One input one output
    
    def forward(self, x):
      
      y_pred = self.Linear(x)
      
      return y_pred

model = Model()

#Construct loss and optimizer
criterion = torch.nn.MSELoss(size_average = False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #Tell what parameters to puddate

#Training loop
for epoch in range(10):
  #Forward pass
  y_pred = model(x_data)
  
  #Compute and print loss
  loss = criterion(y_pred, y_data)
  print(epoch, loss.data[0])
  
  #Zero gradients, perform backprop and update the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step() #Gradient descent step
  
  #Test model
  #model.froward(some data)

	grad:  1 2 -2.0
	grad:  2 4 -7.84
	grad:  3 6 -16.2288
progress: 0 w= 1.260688 loss= <function loss at 0x7f7fc9f74950>
	grad:  1 2 -1.478624
	grad:  2 4 -5.796206079999999
	grad:  3 6 -11.998146585599997
progress: 1 w= 1.453417766656 loss= <function loss at 0x7f7fc9f74950>
	grad:  1 2 -1.093164466688
	grad:  2 4 -4.285204709416961
	grad:  3 6 -8.87037374849311
progress: 2 w= 1.5959051959019805 loss= <function loss at 0x7f7fc9f74950>
	grad:  1 2 -0.8081896081960389
	grad:  2 4 -3.1681032641284723
	grad:  3 6 -6.557973756745939
progress: 3 w= 1.701247862192685 loss= <function loss at 0x7f7fc9f74950>
	grad:  1 2 -0.59750427561463
	grad:  2 4 -2.3422167604093502
	grad:  3 6 -4.848388694047353
progress: 4 w= 1.7791289594933983 loss= <function loss at 0x7f7fc9f74950>
	grad:  1 2 -0.44174208101320334
	grad:  2 4 -1.7316289575717576
	grad:  3 6 -3.584471942173538
progress: 5 w= 1.836707389300983 loss= <function loss at 0x7f7fc9f74950>
	grad:  1 2 -0.3265852213980338
	grad:  2 4 -1.28021406788

In [0]:
#Logistic Regression w/ Pytorch to predict pass fail on study hours
import torch
from torch.autograd import Variable
import torch.nn.functional as F

x_data = [1, 2, 3]
y_data = [2, 4, 6]

class Model(torch.nn.Module):
  
  def __init__(self):
    super(Model, self).__init__()
    self.linear = torch.nn.Linear(1,1)
    
  def forward(self, x):
    y_pred = F.sigmoid(self.linear(x)) #Apply linear layer, then sigmoid
    return y_pred
  
model = Model()

criterion = torch.nn.BCELoss(size_average = True) #BCE Loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10):
  #Forward pass
  y_pred = model(x_data)
  
  #Compute and print loss
  loss = criterion(y_pred, y_data)
  print(epoch, loss.data[0])
  
  #Zero gradients, perform backprop and update the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step() #Gradient descent step
  
 #Test model
house_var = Variable(torch.Tensor([1]))
print("predict 1 hour", 1.0, model(house_var).data[0][0] > 0.5)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


AttributeError: ignored

In [0]:
#Adding layers to Network
torch.nn.Linear([2,1]) #The 2 and 1 mean 2 inputes and one output
#Add layers buy creating multiple lienar components
sigmoid = torch.nn.Sigmoid()
lay1 = torch.nnLinear(2,4) 
lay2 = torch.nn.Linnear(4,3) #The input of this layer has to match
lay3 = torch.nn.Linear(3,1)  #Dimension of previous layer's output
#Combining the layers
out1 = sigmoid(l1(x_data))
out2 = sigmoid(l2(out1))
y_pred = sigmoid(l3(out2))



In [0]:
#Cross entropy soft max example
import numpy as np
Y = np.array([1, 0, 0])

Y_pred1 = np.array([0.7, 0.2, 0.1]) #First predictions, pretty good
Y_pred2 = np.array([0.1, 0.3, 0.6]) #Pretty bad, should have big loss

print("loss=", np.sum(-Y *np.log(Y_pred1))) #0.35
print("loss=", np.sum(-Y *np.log(Y_pred2))) #2.30


#Using built in CrossEntropy 

Y = Variable(torch.LongTensor([0]), requires_grad = False #True label is 0

Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])

print("PyTorch Loss1 = ", l1.data, #0.42
     "\nPyTorch Loss2 =", l2.data) #1.84
